# Regression
1. Regression predictive modeling is the task of approximating a mapping function (f) from input variables(X) to a continuous output variable(y).
2. A continuous output variable is a real-value, such as a n integer or floating point value. These are often quantities, suchas amounts and sizes.
3. For example,a house may be predicted to sell for a specific dollar value, perhaps in the range of $100,000 to $200,000.

## Regression Algorithms
1. Linear regression
2. Support Vector Machines
3. Random forest regression

## Linear Regression
1. Regression is a method of modelling a target value based on independent predictors.
2. This method is mostly used for forecasting and finding out cause and effect relationship between variables.
3. Regression techniques mostly differ based on the number of independent variables and the type of relation ship between the independent and dependent variables.
![Linear Regression](lr.png)

# Training the Model for Analysis

In [7]:
import pandas as pd
import numpy as np

dataset=pd.read_csv('cardio_dataset.csv').values

data=dataset[:,0:7]
target=dataset[:,7]

target=np.reshape(target, (-1,1)) # we provide columns as 1 but rows as -1 means unknown -- so it will make a array of n rows with each value as a single column

print(data[:10])
print(target[:10])
print('size of the dataset:',len(dataset))

[[  2.  48. 236.  66.   0.   2.   0.]
 [  1.  48. 260.  51.   0.   2.   1.]
 [  1.  44. 187.  49.   1.   2.   0.]
 [  2.  42. 216.  57.   1.   2.   0.]
 [  2.  56. 156.  42.   0.   2.   0.]
 [  1.  44. 162.  57.   1.   2.   0.]
 [  1.  50. 244.  47.   0.   2.   0.]
 [  1.  48. 212.  30.   1.   2.   0.]
 [  2.  66. 202.  53.   0.   2.   1.]
 [  1.  63. 186.  46.   1.   2.   0.]]
[[ 1.1]
 [ 7. ]
 [ 7. ]
 [ 0.4]
 [ 2.2]
 [ 3. ]
 [ 4.2]
 [17.4]
 [13.4]
 [17.3]]
size of the dataset: 6644


In [19]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [20]:
from sklearn.linear_model import LinearRegression

model=LinearRegression()

model.fit(train_data,train_target)
predicted_target=model.predict(test_data)

In [21]:
from sklearn.metrics import r2_score

r2=r2_score(test_target,predicted_target)
print("r2 score:",r2)

r2 score: 0.7677084973876125


In [22]:
print('Actual Target:',test_target[:10].T)
print('Predicted Target:',predicted_target[:10].T)

Actual Target: [[12.5 32.3  1.8  4.3 12.2 40.5 40.9  8.2  3.2  0.5]]
Predicted Target: [[17.09511573 28.41053369  3.43936217  0.417671   24.93327013 33.83998292
  32.6735837  13.38437588  3.46912742 -6.03127184]]


In [23]:
import joblib

joblib.dump(model,'heart_risk_prediction_regression_model.sav')

['heart_risk_prediction_regression_model.sav']

# Making the Web Application

### The Main Python file
from flask import Flask,render_template,request
import joblib
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model=joblib.load('heart_risk_prediction_regression_model.sav')

app = Flask(__name__)

@app.route('/',methods=['GET'])
def index():
    return render_template("patient_details.html")

@app.route('/getresults',methods=['GET','POST'])
def results():
    
    result=request.form 
    name=result['name']
    gender=float(result['gender'])
    age=float(result['age'])
    tc=float(result['tc'])
    hdl=float(result['hdl']) 
    sbp=float(result['sbp']) 
    smoke=float(result['smoke'])
    bpm=float(result['bpm'])
    diab=float(result['diab'])
    
    test_data=np.array([gender,age,tc,hdl,smoke,bpm,diab]).reshape(-1,1)
    
    prediction=model.predict(test_data) 
    prediction = max(prediction,0)
    
    resultDict={"name":name,"risk":round(prediction[0][0],2)}
    
    return render_template('patient_results.html',results=resultDict)

if __name__ == "__main__":
    app.run(debug=True)

### The Get Details File
<!DOCTYPE html>
<html>
<head>
	<title>Patient Details</title>
</head>
<body>

	<h1>Heart Disease Risk Predictor</h1>
	<p><b>Please fill this form with your correct and true details</b></p>

	<form action="http://127.0.0.1:5000/getresults" method="POST">
		
		<p>Name <input type="text" name="name"/> </p>
		<p>Gender (1-male,2-Female) <input type="text" name="gender"/> </p>
		<p>Age <input type="text" name="age"/> </p>
		<p>TC(mg/dL) <input type="text" name="tc"/> </p>
		<p>HDL(mg/dL) <input type="text" name="hdl"/> </p>
		<p>SBP(mm Hg) <input type="text" name="sbp"/> </p>
		<p>Smoke(1-yes,0-no) <input type="text" name="smoke"/> </p>
		<p>Blood Pressure Medication(1-Not Taking,2-Taking) <input type="text" name="bpm"/> </p>
		<p>Diabetics(1-yes,0-no) <input type="text" name="diab"/> </p>

		<p><input type="submit" value="Submit Details"/></p>

	</form>

</body>
</html>

![Details](details.jpg)

### The Results File
< !DOCTYPE html><br>
< html><br>
< head><br>
	< title>Results< /title><br>
< /head><br>
< body><br>
	< h1>Results Sheet< /h1><br>
	< p>Name: {{results.name}}< /p><br>
	< p>Risk Level: {{results.risk}}%< /p><br>
< /body><br>
< /html>

![Results](results.jpg)